In [1]:
import os
os.environ["LANGGRAPH_DISABLE_DAG_RENDER"] = "1"
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from google import genai

c:\Users\ANUP N\OneDrive\Desktop\LangGraph Tut\langgraph\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv("GEMINI_API_KEY_SELF") # or os.getenv("GEMINI_API_KEY")
if not api_key:
	raise EnvironmentError("GEMINI_API_KEY2 (or GEMINI_API_KEY) not set in environment. Add it to your .env or export it.")

# client = genai.Client(api_key=api_key)

In [4]:
model = ChatOpenAI(
   model="gemini-2.5-flash",
   api_key=api_key,
   base_url="https://generativelanguage.googleapis.com/v1beta/openai"
)

In [8]:
class Blogstate(TypedDict):
    title: str
    outline: str
    content: str
    rateing: float

In [6]:
def create_outline(state: Blogstate)-> Blogstate:
    title= state['title']
    prompt= f'Generate the detailed outline for a blog on the topic- {title}'
    state['outline']= model.invoke(prompt)
    return state

In [7]:
def create_blog(state: Blogstate) -> Blogstate:
    title= state['title']
    outline= state['outline']
    prompt_blog= f'Write 500 words blog on the title {title} using following outline:\n {outline}'
    content= model.invoke(prompt_blog)
    state['content']= content
    return state

In [13]:
def rate_blog(state: Blogstate) -> Blogstate:
    blog= state['content']
    outline= state['outline']
    title= state['title']
    prompt= f'Rate the blog and provide only the rating in float just float on the scale of 1 to 10, for the title {title} with outline \n {outline} \n \n check if the below blog basis on if it is following the outline properly and blog is : \n {blog}'
    score= model.invoke(prompt)
    state['rateing']= score
    return state

In [14]:
graph= StateGraph(Blogstate)
# nodes
graph.add_node('outline_gen', create_outline )
graph.add_node('generate_blog',create_blog)
graph.add_node('rate_blog', rate_blog)


#edges
graph.add_edge(START, 'outline_gen')
graph.add_edge('outline_gen', 'generate_blog')
graph.add_edge('generate_blog', 'rate_blog')
graph.add_edge('rate_blog', END)

# graph.compile()

In [15]:
workflow= graph.compile()

In [16]:
initial_state= {'title': 'Top 10 movies in 2025 year'}
final_state= workflow.invoke(initial_state)

In [33]:
import markdown
import json

In [30]:
pprint(print(final_state))

Pretty printing has been turned OFF


In [37]:
print(final_state['outline'].content)

Here's a detailed outline for a blog post on "Top 10 Movies in 2025," designed to be engaging, informative, and adaptable given the predictive nature of the topic.

---

## Blog Post Outline: The Most Anticipated Blockbusters & Hidden Gems: Top 10 Movies to Watch in 2025!

**Blog Title Options (Choose one or mix elements):**
*   The Future of Cinema: Our Top 10 Most Anticipated Movies of 2025
*   Beyond the Horizon: Unveiling the Top 10 Must-See Movies of 2025
*   2025 Movie Preview: Your Ultimate Guide to the Year's Top 10 Films
*   Blockbusters, Dramas, and Surprises: The Top 10 Movies Expected to Define 2025

---

### I. Introduction (Approx. 150-200 words)

*   **A. Catchy Hook:** Start with the excitement and anticipation of a new year in cinema.
    *   *Example:* "While 2024 is just getting started (or wrapping up, depending on when this is written), the buzz for what's coming next is already palpable. Hollywood never sleeps, and neither do our expectations for groundbreaking st

In [18]:
print(final_state['content'].content)

## Beyond the Horizon: Unveiling the Top 10 Must-See Movies of 2025!

While 2024 is just getting started, the buzz for what's coming next in cinema is already palpable. Hollywood never sleeps, and neither do our expectations for groundbreaking stories and cinematic masterpieces. Major studios are meticulously planning their slates, beloved franchises are poised for their next chapters, and visionary directors are crafting their magnum opuses. The cyclical nature of big releases ensures that 2025 is already shaping up to be an extraordinary year for film lovers.

Keep in mind, predicting the future of film is a tricky business! This list is based on current announcements, persistent rumors, logical franchise progressions, and the sheer pedigree of talent involved. While release dates and details can shift, these are the films we’re betting on to make significant waves and redefine our cinematic experience in 2025. Get ready for a sneak peek into the year's most anticipated films!

### H

In [20]:
print(final_state['rateing'].content)

9.8
